# CVI620 Midterm Exam - Jupyter Notebook with all tasks and explanation

- **Name:** Aryan Khurana
- **Student ID:** 145282216

## Step 0: Importing Libraries

Here, I am importing libraries that will be used later in the program. I am also initializing a dictionary that maps the name of the images to the path.

In [1]:
import cv2 as cv
import numpy as np

image_hashmap = {
    "Geometric Shapes - OpenCV": "images/geometric_shapes.jpg",
    "Industrial Objects - OpenCV": "images/industrial_objects.jpg",
    "Natural Scenes - OpenCV": "images/natural_scenes.jpg",
}

## Step 1: Acquiring Sample Images

**Objective:** Obtain a set of images for testing edge detection methods. Images can be sourced from natural scenes, industrial objects, or geometric
shapes to highlight various edge properties.

### Explanation of my solution:

Here, I am looping through the image mapping we created earlier and displaying all the images using `cv.imshow()`. I am also appending the images (numpy arrays) to the `images` list so I can use them later in the program.


In [ ]:
images = []

for window_name, image_path in image_hashmap.items():
    image = cv.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image from {image_path}")
        continue
    images.append(image)
    cv.imshow(window_name, image)

cv.waitKey(0)
cv.destroyAllWindows()

## Task 2: Grayscale Conversion

**Objective:** Convert the color images into grayscale to simplify edge detection by focusing on intensity changes.

### Explanation of my solution:

Here, I am creating a list called `gray_images` and then looping through each image in the `images` array that I created earlier. I convert each image to grayscale one by one, append them to the `gray_images` list, and display the grayscale images.

##### Why is grayscale is preferred for edge detection

- Grayscale is preferred for edge detection because it simplifies the image by reducing it to a single channel of intensity values, making the process more efficient.
- Edge detection focuses on identifying sharp changes in intensity between adjacent pixels, and since grayscale images represent only intensity, it highlights these changes without interference from color variations.
- Processing a single grayscale channel also requires less computational power and helps reduce noise by removing unnecessary color information, leading to more accurate and consistent results.

In [3]:
gray_images = []

for i, image in enumerate(images):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray_images.append(gray_image)
    cv.imshow(f"Grayscale - Image {i + 1}", gray_image)

cv.waitKey(0)
cv.destroyAllWindows()

## Task 3: Noise Reduction Using Smoothing

**Objective:** Apply noise reduction techniques to smooth the image and improve edge detection accuracy.

### Explanation of my solution:

Here, I am creating a list called `blurred_images` and then looping through each image in the `gray_images` array. For each grayscale image, I apply a Gaussian blur using `cv.GaussianBlur()` and append the blurred image to the `blurred_images` list.

Additionally, I convert the grayscale images and their blurred counterparts to 3-channel images using `cv.cvtColor()` so that they can be concatenated with the original colored images. I then use `cv.hconcat()` to horizontally concatenate the original image, the grayscale image, and the blurred image into a single output. Finally, I display this concatenated image using `cv.imshow()`, showing the comparison between the original, grayscale, and Gaussian blurred versions.

In [4]:
blurred_images = []
for i in range(len(gray_images)):
    gaussian_blur = cv.GaussianBlur(gray_images[i], (5, 5), 0)
    blurred_images.append(gaussian_blur)

    gray_image_3channel = cv.cvtColor(gray_images[i], cv.COLOR_GRAY2BGR)
    gaussian_blur_3channel = cv.cvtColor(gaussian_blur, cv.COLOR_GRAY2BGR)

    concatenated_image = cv.hconcat(
        [images[i], gray_image_3channel, gaussian_blur_3channel]
    )
    cv.imshow(
        f"Original vs Grayscale vs Gaussian Blurred - Image {i + 1}", concatenated_image
    )

cv.waitKey(0)
cv.destroyAllWindows()


## Task 4: Sobel Edge Detection

**Objective:** Detect edges using the Sobel operator, which calculates the gradient in both the x and y directions.

### Explanation of My Solution:

In this segment, I loop through each image in the `blurred_images` list and apply Sobel edge detection to identify both horizontal and vertical edges. I experiment with three different kernel sizes: `3`, `5`, and `7`.

For each kernel size, I compute the gradient of the image along the x-axis using `cv.Sobel()` with parameters `1, 0` and along the y-axis with parameters `0, 1`. This generates two separate gradient images: `sobel_x` for horizontal edges and `sobel_y` for vertical edges.

Next, I combine these gradients by taking the weighted sum of the absolute values of both `sobel_x` and `sobel_y` using `cv.addWeighted()`, resulting in a combined image, `sobel_combined`, that shows the overall edges in the image. Finally, I convert the combined result to an 8-bit format using `np.uint8()` and display the Sobel edge-detected image with `cv.imshow()`.

### Sobel Edge Detection and Kernel Sizes

Sobel edge detection identifies edges in a picture by leveraging the pixel intensity along both the horizontal (x-axis) and vertical (y-axis) directions.
Larger kernels provide a smoother, more generalized view of edges but may miss finer details, while smaller kernels can capture intricate edges at the risk of introducing noise.

**Experimenting with Kernel Sizes**:

   - The kernel size in the Sobel operator determines how much surrounding pixel information is considered when calculating gradients. A larger kernel size captures more context from the image, leading to smoother gradients but potentially blurring smaller features, while a smaller kernel focuses on fine details.
   
   - **Larger Kernel Sizes**:
     - A larger kernel size (e.g., 7x7 or 9x9) tends to smooth out noise even more and can help identify broader edges more clearly.
     - However, this might cause smaller edges or details to be overlooked or blended into larger features. Consequently, smaller objects or intricate edges may not be detected as clearly, resulting in a loss of detail in the edge map.
   
   - **Smaller Kernel Sizes**:
     - A smaller kernel size (e.g., 3x3 or 5x5) is better suited for detecting fine edges and intricate details within the image. It captures rapid changes in intensity more effectively, making it easier to identify small features.
     - The downside is that smaller kernels are more sensitive to noise, which can introduce false edges or artifacts in the resulting edge map. This might lead to an edge map that appears cluttered or noisy.


In [11]:
kernel_sizes = [3, 5, 7]
sobel_images = []

for index, blurred_image in enumerate(blurred_images):
    for ksize in kernel_sizes:
        sobel_x = cv.Sobel(blurred_image, cv.CV_64F, 1, 0, ksize=ksize)
        sobel_y = cv.Sobel(blurred_image, cv.CV_64F, 0, 1, ksize=ksize)

        sobel_combined = cv.addWeighted(np.abs(sobel_x), 0.5, np.abs(sobel_y), 0.5, 0)

        sobel_combined = np.uint8(sobel_combined)
        
        if ksize == 3:
            sobel_images.append(sobel_combined)
            
        window_name = f"Sobel Edge Detection - Kernel Size: {ksize}"
        cv.imshow(window_name, sobel_combined)

cv.waitKey(0)
cv.destroyAllWindows()


## Task 5: Laplacian Edge Detection

**Objective:** Detect edges using the Laplacian operator, which highlights regions of rapid intensity change in an image.

### Explanation of My Solution:

In this segment, I loop through each image in the `blurred_images` list and apply the Laplacian edge detection technique to identify edges based on intensity gradients. The Laplacian operator computes the second derivative of the image intensity, effectively measuring the rate of change of the gradient.

The Laplacian operator is particularly effective at highlighting edges because it responds to regions where there is a rapid change in intensity, regardless of the direction of the change. Unlike the Sobel operator, which calculates gradients in specific directions (horizontal and vertical), the Laplacian operator provides a more generalized approach by considering the second derivative. 

#### Laplacian Second Derivative v/s Sobel Gradient Calculation

- The Laplacian operator calculates the second derivative of image intensity, making it sensitive to noise and effective at detecting edges regardless of direction, which can result in a cluttered output with false edges. 
- In contrast, the Sobel operator computes the first derivative in both horizontal and vertical directions, producing clearer and more defined edges with reduced noise sensitivity due to its averaging of pixel intensities. 
- While the Laplacian can highlight a broader range of intensity changes, Sobel is often preferred in practical applications for its robustness and directional sensitivity, making it particularly effective in scenarios requiring precise feature extraction and object detection.

In [12]:
laplacian_images = []

for index, blurred_image in enumerate(blurred_images):
    laplacian = cv.Laplacian(blurred_image, cv.CV_64F)
    laplacian = np.abs(laplacian)
    laplacian = np.uint8(laplacian)
    laplacian_images.append(laplacian)
    cv.imshow(f"Laplacian Edge Detection - Image {index + 1}", laplacian)

cv.waitKey(0)
cv.destroyAllWindows()

In this code snippet, I iterate through pairs of Sobel and Laplacian edge-detected images, converting each to a three-channel format for display. I print their shapes and data types, then ensure the Laplacian image matches the Sobel image's dimensions by resizing if necessary. Finally, I concatenate the two images horizontally and display them in a window labeled "Sobel vs Laplacian - Image X" for visual comparison of their edge detection results.

In [13]:
for i in range(len(sobel_images)):
    sobel_image_3channel = cv.cvtColor(sobel_images[i], cv.COLOR_GRAY2BGR)
    laplacian_image_3channel = cv.cvtColor(laplacian_images[i], cv.COLOR_GRAY2BGR)

    if laplacian_image_3channel.shape[:2] != sobel_image_3channel.shape[:2]:
        laplacian_image_3channel = cv.resize(laplacian_image_3channel, (sobel_image_3channel.shape[1], sobel_image_3channel.shape[0]))

    concatenated_image = cv.hconcat([sobel_image_3channel, laplacian_image_3channel])
    cv.imshow(f"Sobel vs Laplacian - Image {i + 1}", concatenated_image)


cv.waitKey(0)
cv.destroyAllWindows()

## Task 7: Canny Edge Detection

**Objective:** Implement the Canny edge detection algorithm, which is a multi-stage process for detecting strong and weak edges.

Here's an explanation of your Canny edge detection code:

### Explanation of My Solution

In this segment, I loop through each image in the `blurred_images` list and apply the Canny edge detection technique with different threshold values, stored in the `thresholds` list. The thresholds determine the sensitivity of the edge detector, with lower values detecting more edges (including noise) and higher values focusing only on stronger edges.

I define three different pairs of thresholds: `(50, 150)`, `(100, 200)`, and `(150, 250)`, to explore the impact of these values on the edge detection results.

For each image, I apply the `cv.Canny()` function using each threshold pair and display the result using `cv.imshow()`. However, I limit the `canny_images` list to store only the first 3 images produced. This allows me to experiment with all the threshold values for every image, but only keep a small subset of the results for further analysis and comparison.

Finally, I display all the Canny-detected edges for each image, and once the comparisons are done, I clean up using `cv.waitKey(0)` and `cv.destroyAllWindows()`.

### Canny Edge Detection and Threshold Values

Canny edge detection identifies edges by calculating the intensity gradient of an image and tracing the edges where intensity changes are the most prominent. The edge detection is controlled by two threshold values: a lower threshold that identifies weak edges and a higher threshold for strong edges. 

**Experimenting with Threshold Values**:

   - The threshold values in the Canny operator determine the sensitivity of edge detection. Lower thresholds detect more edges, including weaker ones, which may include noise. Higher thresholds, on the other hand, detect only stronger, more prominent edges, potentially missing finer details.

   - **Lower Thresholds**:
     - A lower threshold (e.g., 50) makes the algorithm more sensitive to edges, detecting even faint or weak edges. This can be helpful in images where fine details are important, but it may also result in more noise or false edges being detected.
     - The risk of using lower thresholds is the introduction of noise, as it can mistake small changes in intensity for meaningful edges.
   
   - **Higher Thresholds**:
     - A higher threshold (e.g., 150) makes the edge detector focus on more prominent edges, reducing noise and clutter. This results in a cleaner edge map but can potentially overlook weaker edges or finer details.
     - The downside of higher thresholds is that smaller or less distinct edges might be missed, leading to a more simplified and less detailed edge map.
    
### Canny v/s Sobel v/s Laplacian

- **Canny edge detection** is considered the most accurate because it uses a series of steps to effectively identify edges. It first smooths the image to reduce noise, which helps prevent false edges from appearing. Then, it finds areas of rapid intensity change and refines these edges to make them sharp and well-defined. However, this process requires more computation, making it slower.
- **Sobel edge detection**, on the other hand, is faster and simpler. It focuses on calculating how quickly pixel values change in both horizontal and vertical directions. While this method can quickly identify edges, it may not be as precise because it can blend thinner edges together or get confused by noise in the image.
- **Laplacian edge detection** highlights regions of rapid intensity change but is more sensitive to noise, which can lead to more false edges. It’s quicker than Canny but may produce less accurate results.

> **Overall, Canny is great for accuracy but takes longer, while Sobel and Laplacian are faster but might not catch all the details as clearly.**

In [16]:
canny_images = []

thresholds = [(50, 150), (100, 200), (150, 250)] 

for index, blurred_image in enumerate(blurred_images):
    for low_thresh, high_thresh in thresholds:
        canny_edges = cv.Canny(blurred_image, low_thresh, high_thresh)

        if low_thresh == 50:
            canny_images.append(canny_edges)

        cv.imshow(f"Canny Edge Detection - Image {index + 1} (Thresh: {low_thresh}, {high_thresh})", canny_edges)

cv.waitKey(0)
cv.destroyAllWindows()


In this code, I loop through the `sobel_images`, `laplacian_images`, and `canny_images` lists, ensuring that each image from the Laplacian and Canny methods is resized to match the dimensions of the corresponding Sobel image. Then, I concatenate the Sobel, Laplacian, and Canny edge-detected images horizontally for each index, displaying the combined result in a single window for side-by-side comparison of the different edge detection methods.

In [ ]:
for i in range(len(sobel_images)):
    sobel_image = sobel_images[i]
    laplacian_image = laplacian_images[i]
    canny_image = canny_images[i]
    height, width = sobel_image.shape[:2]

    if laplacian_image.shape[:2] != (height, width):
        laplacian_image = cv.resize(laplacian_image, (width, height))

    if canny_image.shape[:2] != (height, width):
        canny_image = cv.resize(canny_image, (width, height))

    concatenated_image = cv.hconcat([sobel_image, laplacian_image, canny_image])
    
    cv.imshow(f"Sobel vs Laplacian vs Canny - Image {i + 1}", concatenated_image)

cv.waitKey(0)
cv.destroyAllWindows()